# **EcoSortify (sortibot-qa-1.0beta) Chatbot Evaluation using Exact Match and F1 Score Metric**

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.6 MB/s eta 0:00:00


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os

from google import genai
from google.genai import types

PROJECT_ID = "ecosortify-459004"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
# Vertex AI SDK
import vertexai
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils

# Data manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
experiment_name = 'projects/838338694702/locations/us-central1/metadataStores/default/contexts/tuning-experiment-20250525033333956401'
tuned_model_endpoint = 'projects/838338694702/locations/us-central1/endpoints/1199379169415266304'

In [ ]:
!gsutil cp gs://ecosortify-assets/qna-pairs/raw_guide1_qna_pairs.csv .

Copying gs://ecosortify-assets/qna-pairs/raw_guide1_qna_pairs.csv...
/ [1 files][ 45.9 KiB/ 45.9 KiB]                                                
Operation completed over 1 objects/45.9 KiB.                                     


In [ ]:
guide1_df = pd.read_csv('raw_guide1_qna_pairs.csv')
guide1_df

,question,answer
0,Apa yang terjadi pada materi menurut hukum kek...,"Menurut hukum kekekalan materi, materi tidak d..."
1,Kapan sebuah benda yang diproduksi untuk kebut...,Sebuah benda berubah menjadi sampah ketika ia ...
2,Mengapa sampah dianggap sebagai material yang ...,Sampah dianggap demikian karena dalam konteks ...
3,Faktor apa yang mempercepat terbentuknya sampa...,Pola konsumsi dan produksi manusia yang terus ...
4,Apakah benda yang dibuang oleh manusia benar-b...,"Tidak, ketika manusia membuang sesuatu, benda ..."
...,...,...
173,"Apa dampak dari tidak terkelolanya 22,3 juta t...","Tidak terkelolanya 22,3 juta ton sampah terseb..."
174,Mengapa rendahnya kesadaran masyarakat menjadi...,Karena jika masyarakat tidak sadar akan pentin...
175,"Jika sampah tidak dipilah dari rumah, apa yang...","Jika tidak dipilah dari rumah, material daur u..."
176,Apa bahaya utama dari gas dioksin dan furan ya...,Dioksin dan furan adalah senyawa kimia yang sa...


In [ ]:
system_instruction = f"""Kamu adalah AI yang mengkhususkan diri dalam menjawab pertanyaan seputar edukasi pengetahuan dan pemilahan sampah. Tujuan kamu adalah memberikan jawaban yang edukatif, jelas, dan mudah dipahami oleh pengguna umum dalam bahasa Indonesia.
Jawaban kamu harus berbobot dan komprehensif, tetapi tetap ringkas dan tidak terlalu kaku atau terlalu formal. Gunakan gaya bahasa yang ramah, khas seperti chatbot yang siap membantu pengguna.
Gunakan konteks dari pertanyaan yang saya berikan untuk memberikan jawaban terbaik. Jawaban tidak perlu terlalu panjang, tapi harus mengandung informasi yang akurat dan bermanfaat.
Jika saya memberikan pertanyaan, cukup jawab langsung sesuai instruksi di atas. Tapi jika diperlukan, berikan tambahan penjelasan atau format khusus lainnya.
"""

In [ ]:
import time

def get_predictions(question: str, model_version: str) -> str:

    time.sleep(5)

    prompt = question
    base_model = model_version

    response = client.models.generate_content(
        model=base_model,
        contents=prompt,
        config={
            "system_instruction": system_instruction,
            "temperature": 1,
            "max_output_tokens": 8192,
            "top_p": 0.95,
        },
    )

    return response.text

In [ ]:
test_df = guide1_df.sample(n=50)

test_df["predicted_answer"] = test_df["question"].apply(get_predictions, model_version=tuned_model_endpoint)
test_df.head(5)

,question,answer,predicted_answer
111,Apa dasar hukum utama dalam pengelolaan sampah...,Dasar hukum utama dalam pengelolaan sampah di ...,Undang-Undang Nomor 18 Tahun 2008 tentang Peng...
152,Sebutkan satu contoh pelanggaran terhadap UU P...,Membuang sampah sembarangan atau tidak melakuk...,Salah satu contohnya adalah ketika seseorang d...
20,"Selain mencegah pencemaran, apa manfaat lain d...",Pengetahuan ini juga membantu kita mengurangi ...,"Meningkatkan rasa tanggung jawab, kreativitas,..."
52,Siapa yang berperan dalam proses penguraian sa...,Proses penguraian sampah organik biasanya meli...,Proses penguraian sampah organik melibatkan ak...
169,Apa perbedaan antara limbah infeksius dan limb...,Limbah infeksius adalah sampah yang terkontami...,Limbah infeksius adalah sampah yang mengandung...


In [ ]:
from collections import Counter

def normalize_answer(s):
    """Lower text and remove extra whitespace, but preserve newlines."""

    def white_space_fix(text):
        return " ".join(text.split())  # Splits by any whitespace, including \n

    def lower(text):
        return text.lower()

    return white_space_fix(lower(s))

def f1_score_squad(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def calculate_em_and_f1(y_true, y_pred):
    """Calculates EM and F1 scores for DataFrame columns."""

    # Ensure inputs are Series
    if not isinstance(y_true, pd.Series):
        y_true = pd.Series(y_true)
    if not isinstance(y_pred, pd.Series):
        y_pred = pd.Series(y_pred)

    em = np.mean(y_true.combine(y_pred, exact_match_score))
    f1 = np.mean(y_true.combine(y_pred, f1_score_squad))

    # # Print non-matching pairs (using index for clarity)
    # for i, (t, p) in enumerate(zip(y_true, y_pred)):
    #     if not exact_match_score(p, t):
    #         print(f"No EM Match at index {i}:\nTrue: {t}\nPred: {p}\n")

    return em, f1

In [ ]:
test_df["predicted_answer"] = test_df["predicted_answer"].apply(normalize_answer)

In [ ]:
em, f1 = calculate_em_and_f1(
    test_df["answer"],
    test_df["predicted_answer"]
)
print(f"EM score: {em}")
print(f"F1 score: {f1}")

EM score: 0.803456664533455
F1 score: 0.897243884139517
